<a href="https://colab.research.google.com/github/MALAKBADER00/Future-Options-Explorer/blob/main/fake_news_classifier_bidirectionalLSTM_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf

In [ ]:
import pandas as pd

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')
df_test= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test.csv')

In [ ]:
df_train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df_test.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [ ]:
print(df_train.shape)
df_train.isnull().sum()

(18285, 5)


id        0
title     0
author    0
text      0
label     0
dtype: int64

In [ ]:
df_train = df_train.dropna()
df_train.shape

(18285, 5)

In [ ]:
# get independent features
X = df_train.drop('label', axis=1)
X

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [ ]:
# independent feature
y = df_train['label']
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 18285, dtype: int64

In [ ]:
# check balance of data set
y.value_counts()
# not balanced

label
0    10361
1     7924
Name: count, dtype: int64

In [ ]:
tf.__version__

'2.15.0'

In [ ]:
from tensorflow.keras.layers import Embedding , LSTM , Dense , Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text  import one_hot

In [ ]:
voc_size = 5000

In [ ]:
messages = X.copy()
messages.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# Data preprocressing
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(messages)):
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review =review.lower()
  review = review.split()

  review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords.words('english'))]
  review = " ".join(review)
  corpus.append(review)

In [ ]:
onehot_rep = [one_hot(words,voc_size) for words in corpus]
onehot_rep

[[4108, 4948, 4146, 633, 3897, 1496, 1918, 3078, 4585, 2280],
 [1054, 41, 3622, 4446, 2498, 2207, 3780],
 [2992, 4095, 4111, 671],
 [3851, 2142, 4779, 1262, 2052, 257],
 [851, 2498, 3592, 825, 4203, 581, 2498, 4867, 3837, 1358],
 [4105,
  895,
  3213,
  110,
  859,
  694,
  2958,
  3932,
  1579,
  1049,
  693,
  1486,
  4147,
  3475,
  3780],
 [2106, 2891, 4836, 744, 628, 1634, 713, 4422, 3636, 3928, 2074],
 [490, 1095, 4275, 4641, 3281, 4558, 694, 439, 3636, 3928, 2074],
 [451, 803, 3307, 833, 3461, 2762, 850, 4079, 694, 3837],
 [4974, 685, 66, 726, 64, 860, 4469, 3528],
 [3101, 3902, 710, 3451, 2665, 3126, 4893, 1725, 2126, 1254, 1703],
 [1262, 1829, 3897, 3454, 694, 3281],
 [2686, 2437, 410, 2302, 3337, 2960, 3942, 2384, 22],
 [1622, 1931, 4419, 3792, 320, 4019, 2780, 3636, 3928, 2074],
 [2023, 3023, 2371, 1923, 145, 3636, 3928, 2074],
 [2352, 1422, 1592, 5, 4084, 279, 2285, 575, 2907, 4614],
 [1511, 2752, 41],
 [3643, 927, 2613, 2563, 694, 4583, 3892, 3780],
 [4588, 3559, 3622, 243

In [ ]:
# Empeding representation
sent_length = 20
embedded_docs = pad_sequences(onehot_rep, padding = 'pre', maxlen= sent_length)

embedded_docs

array([[   0,    0,    0, ..., 3078, 4585, 2280],
       [   0,    0,    0, ..., 2498, 2207, 3780],
       [   0,    0,    0, ..., 4095, 4111,  671],
       ...,
       [   0,    0,    0, ..., 3636, 3928, 2074],
       [   0,    0,    0, ...,  289, 2405, 1711],
       [   0,    0,    0, ..., 3867, 4917, 1093]], dtype=int32)

In [ ]:
#Creating a model
embedding_vector_features =40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length = sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirection  (None, 200)               112800    
 al)                                                             
                                                                 
 dense_3 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313001 (1.19 MB)
Trainable params: 313001 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [ ]:
from xml.sax.saxutils import XMLFilterBase
#train test split
from sklearn.model_selection import train_test_split
X_train ,X_test , y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state = 100)

In [ ]:
# model training
model.fit(X_train, y_train, validation_data =(X_test,y_test), epochs= 10, batch_size = 32)

Epoch 1/10
383/383 [==============================] - 21s 42ms/step - loss: 0.2661 - accuracy: 0.8778 - val_loss: 0.1902 - val_accuracy: 0.9173
Epoch 2/10
383/383 [==============================] - 5s 12ms/step - loss: 0.1329 - accuracy: 0.9487 - val_loss: 0.2259 - val_accuracy: 0.9123
Epoch 3/10
383/383 [==============================] - 4s 9ms/step - loss: 0.0825 - accuracy: 0.9710 - val_loss: 0.2323 - val_accuracy: 0.9201
Epoch 4/10
383/383 [==============================] - 4s 10ms/step - loss: 0.0476 - accuracy: 0.9841 - val_loss: 0.3262 - val_accuracy: 0.9170
Epoch 5/10
383/383 [==============================] - 5s 13ms/step - loss: 0.0309 - accuracy: 0.9903 - val_loss: 0.3603 - val_accuracy: 0.9080
Epoch 6/10
383/383 [==============================] - 3s 9ms/step - loss: 0.0178 - accuracy: 0.9945 - val_loss: 0.3984 - val_accuracy: 0.9084
Epoch 7/10
383/383 [==============================] - 4s 9ms/step - loss: 0.0117 - accuracy: 0.9961 - val_loss: 0.4467 - val_accuracy: 0.9075
E

In [ ]:
y_pred= model.predict(X_test)

189/189 [==============================] - 1s 3ms/step


In [ ]:
y_pred = np.where(y_pred>=0.5,1,0)

In [ ]:
from sklearn.metrics import confusion_matrix , accuracy_score
confusion_matrix(y_test,y_pred)

array([[3182,  278],
       [ 273, 2302]])

In [ ]:
accuracy_score(y_test,y_pred)

0.9086992543496272